In [95]:
import json
import os
import pandas as pd
import librosa
import audioread
import numpy as np
import resampy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
import nltk
import warnings
import urllib.request
from konlpy.tag import Okt
warnings.filterwarnings('ignore')

In [96]:
# 일반 음성 데이터 불러오기 및 라벨링

os.chdir('/Users/medici/Downloads/json')
text_dir = '/Users/medici/Downloads/json/'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass


d = pd.DataFrame(d)
normal_dir = '/Users/medici/Downloads/일반대화 음성/'
normal_files = os.listdir(normal_dir)
d['wav'] = normal_files
normal= d
normal.rename(columns={0:'txt'},inplace=True)
normal['voice_fishing']=0
normal

,txt,wav,voice_fishing
0,내가 나한테 잘해주면 돼요.혹시 여러분들 자식들가운데?지금 손자 손녀키운다고 먹고 ...,1-converted.ogg,0
1,운명을 바꿔 놓습니다.서울대 음대를 나왔는데요.서울대가 중요한 게 아닙니다.결국 이...,10-converted.ogg,0
2,"네, 감사합니다.네, 감사합니다.서울대 면접없는 판 모국입니다 검사대 여름말씀이신건...",100-converted.ogg,0
3,그러나 인류는 절대 내가 내 말에 절대 라는 용어를 여간 해서는 절대 안 된다는 얘...,11-converted.ogg,0
4,그런데 외국사람이결국은 집 이사 이제 세방을 얻어 는데 그때 50 전세 한 달에 5...,12-converted.ogg,0
...,...,...,...
95,회원 가입을 했거든요 하신다하는 건가요?혹시 방법을 높여야 하는지 잘 몰라서 보는데...,95-converted.ogg,0
96,근데 이제 장소가 이제 구애가 없다고 하시면 저희가 그을지로 고용노동부건물에 있는 ...,96-converted.ogg,0
97,감사합니다.확인 도와드릴 텐데 방송한 기다려 주시겠어요?여기야 기자 감사드리고요.거...,97-converted.ogg,0
98,네 감사합니다.저 인사담당자신청했는데요 그 a니까 아직도 않았어요?네 감사합니다.,98-converted.ogg,0


In [97]:
# 보이스피싱 데이터 불러오기 및 라벨링 

os.chdir('/Users/medici/Downloads/보이스피싱 json')
text_dir = '/Users/medici/Downloads/보이스피싱 json/'

voice_spam_text = [f for f in os.listdir()]
# JSON 파일 열기
d = []
for k,file in enumerate(voice_spam_text):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    d.append('')
    for i in range(len(data['Transcript'])):
        try:
            d[k]= d[k]+data['Transcript'][2*i-1]['Content']
        except IndexError:
            pass
d = pd.DataFrame(d)
voice_dir = '/Users/medici/Downloads/보이스피싱 음성/'
voice_files = os.listdir(voice_dir)
d['wav'] = voice_files
voice= d
voice.rename(columns={0:'txt'},inplace=True)
voice['voice_fishing']=1

voice

,txt,wav,voice_fishing
0,지금 전국적으로 약 이백 여명이 넘게 연료가 되어 있는 사건이고 일단 저희도 어느 ...,1-1-converted.ogg,1
1,금융사기사건이었습니다네.직접 방문하셔서 개설하신.금융권은 어디 금융권들이 있으시죠?...,1-2-converted.ogg,1
2,그럼 최근 삼 년 사이에 본인개인정보가 담긴 물품 분실하신 적 전혀 없으십니까?제가...,1-3-converted.ogg,1
3,사실 있는 그대로만 진술하시면 되시는데 현재 계신 곳이 직장 이십니까.통합 불편하시...,1-4-converted.ogg,1
4,일단 이 두 통장은 범죄로 사용돼서 추가적인 피해자가 발생하지 않도록 본인 의사와 ...,1-5-converted.ogg,1
...,...,...,...
77,많은 분들이 연루돼 있는 부분들가운데서 저희가 지금 동시에 수사를 진행하다 보니까 ...,9-4.wav,1
78,이렇게 지금 전화 받아 가지고 있던 말아무도 본인이 지금 왜 보려는 전에 는지 본회...,9-5.wav,1
79,이제 최종 처리를 포함한 공범들하고는 이제 발견된 연관성이 없었기 때문에 일차적인 ...,9-6.wav,1
80,사건 으로 외모되셨으면 확인 한번 부탁드립니다.네 오세요.서울중앙지검에 김진수 아닙...,9-7.wav,1


In [98]:
df = pd.concat([voice,normal],axis=0)
df.reset_index(drop=True,inplace=True)

In [99]:
# wav 데이터 mfcc로 수치화

voice_spam_dir = '/Users/medici/Downloads/보이스피싱 음성/'
normal_dir = '/Users/medici/Downloads/일반대화 음성/'
df['filepath']=''
df['voice_feature']=''


for i in range(len(df)):
    if df.at[i,'voice_fishing']==1:
        df.at[i,'filepath'] = os.path.join(voice_spam_dir, df.at[i,'wav'])
        y, sr = librosa.load(df.at[i,'filepath'], sr=22050)
        y = librosa.util.normalize(y)  # 음량 조정
        mfccs1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs_mean1 = np.mean(mfccs1, axis=1)
        df.at[i,'voice_feature'] = mfccs_mean1

    else:
        df.at[i,'filepath'] = os.path.join(normal_dir, df.at[i,'wav'])
        y, sr = librosa.load(df.at[i,'filepath'], sr=22050)
        y = librosa.util.normalize(y)  # 음량 조정
        mfccs1 = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        mfccs_mean1 = np.mean(mfccs1, axis=1)
        df.at[i,'voice_feature'] = mfccs_mean1

In [100]:
# txt 데이터 TF-IDF로 수치화

temp=list(df['txt'])

df['txt_feature']=''
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=50,lowercase=False) # TF-IDF 상위 50개 추출
for i in range(len(df)):
    df.at[i,'txt_feature'] = vectorizer.fit_transform(temp)[i]

a=pd.DataFrame()
for i in range(len(df)):
    b=pd.DataFrame(df.at[i,'txt_feature'].todense())
    a=pd.concat([a,b],axis=0)
a.reset_index(drop=True,inplace=True)
df =pd.concat([df,a],axis=1)
df=df.drop(columns='txt_feature')

df.head(2)

,txt,wav,voice_fishing,filepath,voice_feature,0,1,2,3,4,...,40,41,42,43,44,45,46,47,48,49
0,지금 전국적으로 약 이백 여명이 넘게 연료가 되어 있는 사건이고 일단 저희도 어느 ...,1-1-converted.ogg,1,/Users/medici/Downloads/보이스피싱 음성/1-1-converted...,"[-329.98514, 126.196266, -32.76896, 11.3803215...",0.0,0.0,0.0,0.0,0.000000,...,0.150224,0.049030,0.287916,0.235100,0.00000,0.000000,0.000000,0.051535,0.0,0.056934
1,금융사기사건이었습니다네.직접 방문하셔서 개설하신.금융권은 어디 금융권들이 있으시죠?...,1-2-converted.ogg,1,/Users/medici/Downloads/보이스피싱 음성/1-2-converted...,"[-316.93973, 157.07793, -23.155636, 8.142115, ...",0.0,0.0,0.0,0.0,0.476662,...,0.000000,0.140148,0.000000,0.224003,0.18568,0.303455,0.169637,0.000000,0.0,0.000000


In [101]:
# TF-IDF 상위 50개 값 확인

vectorizer = TfidfVectorizer(max_features=50, lowercase=False)
# 보이스피싱 텍스트에서 상위 50개의 tf-idf 텍스트와 해당 값출력
voice_tfidf = vectorizer.fit_transform(temp)
feature_names = vectorizer.get_feature_names()
indices = np.argsort(vectorizer.idf_)[::-1]
top_n = 50
top_features = [feature_names[i] for i in indices]
top_scores = [vectorizer.idf_[i] for i in indices]
print("보이스피싱 중요도 TOP50 :")
for i in range(top_n):
    print(f"{top_features[i]}: {top_scores[i]}")

보이스피싱 중요도 TOP50 :
내가: 2.8421903228549468
우리: 2.8082887711792655
본인께서: 2.6541380913520074
연락을: 2.5718999931150353
이거: 2.5206066987274847
저는: 2.4718165345580525
직접: 2.4718165345580525
그러니까: 2.4482860371478585
됩니다: 2.4482860371478585
오늘: 2.42529651892316
본인이: 2.42529651892316
그런데: 2.42529651892316
되어: 2.402823663071101
바로: 2.3382851419335298
가지고: 2.3382851419335298
어떻게: 2.297463147413275
그런: 2.277660520117095
그러면: 2.277660520117095
많이: 2.2582424342599934
본인: 2.2582424342599934
하는: 2.2582424342599934
대해서: 2.2582424342599934
어떤: 2.239194239289299
가지: 2.239194239289299
이게: 2.220502106277147
그냥: 2.220502106277147
다른: 2.20215296760895
있어요: 2.1841344621062717
해서: 2.1841344621062717
일단: 2.1841344621062717
같은: 2.166434885006871
혹시: 2.166434885006871
있습니다: 2.11514159061932
겁니다: 2.11514159061932
근데: 2.0823517677963297
되는: 2.0350988829457837
이런: 2.0350988829457837
하고: 2.0198314108149953
거예요: 1.960990910792062
한번: 1.960990910792062
저희: 1.9054210596372512
저희가: 1.865680730987737
그리고: 1.8527773261518292

In [103]:
# 음성 데이터 학습

df_temp=df.drop(columns=['txt','wav','voice_fishing','filepath'])
X = df_temp
y = df['voice_fishing']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.5,random_state= 18)
X_train1=list(X_train['voice_feature'])
X_test1=list(X_test['voice_feature'])
y_train1 = list(y_train)
y_test1 = list(y_test)
print(len(X_train1))
print(len(y_train1))

# randomforest
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
rf.fit(X_train1, y_train1)
y_pred_rf = rf.predict(X_test1)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test1, y_pred_rf))
print("Recall:", recall_score(y_test1, y_pred_rf))
print(' ')
# gradient boosting
gb = GradientBoostingClassifier(n_estimators=100)
gb.fit(X_train1, y_train1)
y_pred_gb = gb.predict(X_test1)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test1, y_pred_gb))
print("Recall:", recall_score(y_test1, y_pred_gb))
print(' ')
# xgboost
xgb = XGBClassifier(n_estimators=100,n_jobs=-1)
xgb.fit(X_train1, y_train1)
y_pred_xgb = xgb.predict(X_test1)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test1, y_pred_xgb))
print("Recall:", recall_score(y_test1, y_pred_xgb))
print(' ')
# Lightgbm
lgbm = LGBMClassifier(n_estimators=100,n_jobs=-1)
lgbm.fit(X_train1, y_train1)
y_pred_lgbm = lgbm.predict(X_test1)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test1, y_pred_lgbm))
print("Recall:", recall_score(y_test1, y_pred_lgbm))

91
91
RandomForest:
Accuracy: 0.8681318681318682
Recall: 0.8095238095238095
 
GradientBoosting:
Accuracy: 0.9230769230769231
Recall: 0.8571428571428571
 
XGBoost:
Accuracy: 0.9230769230769231
Recall: 0.8571428571428571
 
LightGBM:
Accuracy: 0.9230769230769231
Recall: 0.8809523809523809


In [104]:
# 음성 모델 ensemble

ensemble_wav = VotingClassifier(estimators=[('rf', rf), ('gb', gb), ('xgb', xgb), ('lgbm', lgbm)], voting='soft')
ensemble_wav.fit(X_train1, y_train1)
y_pred1 = ensemble_wav.predict(X_test1)

print("ensemble Recall:", recall_score(y_test1, y_pred1))
print("Accuracy:", accuracy_score(y_test1, y_pred1))

ensemble Recall: 0.8809523809523809
Accuracy: 0.9340659340659341


In [105]:
# 텍스트 데이터 학습

X_train2=X_train.drop(columns='voice_feature')
X_test2=X_test.drop(columns='voice_feature')
y_train2 = list(y_train)
y_test2 = list(y_test)
print(len(X_train2))
print(len(y_train2))

# randomforest
rf_text = RandomForestClassifier(n_estimators=100,n_jobs=-1, max_features=50)
rf_text.fit(X_train2, y_train2)
y_pred_rf = rf_text.predict(X_test2)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test2, y_pred_rf))
print("Recall:", recall_score(y_test2, y_pred_rf))
print(' ')
# gradient boosting
gb_text = GradientBoostingClassifier(n_estimators=100)
gb_text.fit(X_train2, y_train2)
y_pred_gb = gb_text.predict(X_test2)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test2, y_pred_gb))
print("Recall:", recall_score(y_test2, y_pred_gb))
print(' ')
# xgboost
xgb_text = XGBClassifier(n_estimators=100,n_jobs=-1)
xgb_text.fit(X_train2, y_train2)
y_pred_xgb = xgb_text.predict(X_test2)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test2, y_pred_xgb))
print("Recall:", recall_score(y_test2, y_pred_xgb))
print(' ')
# Lightgbm
lgbm_text = LGBMClassifier(n_estimators=100,n_jobs=-1)
lgbm_text.fit(X_train2, y_train2)
y_pred_lgbm = lgbm_text.predict(X_test2)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test2, y_pred_lgbm))
print("Recall:", recall_score(y_test2, y_pred_lgbm))

91
91
RandomForest:
Accuracy: 0.9230769230769231
Recall: 0.9047619047619048
 
GradientBoosting:
Accuracy: 0.8681318681318682
Recall: 0.8333333333333334
 
XGBoost:
Accuracy: 0.9340659340659341
Recall: 0.9047619047619048
 
LightGBM:
Accuracy: 0.9230769230769231
Recall: 0.9047619047619048


In [106]:
# 텍스트 모델 ensemble
ensemble_txt = VotingClassifier(estimators=[('rf', rf_text), ('gb', gb_text), ('xgb', xgb_text), ('lgbm', lgbm_text)], voting='soft')
ensemble_txt.fit(X_train2, y_train2)
y_pred2 = ensemble_txt.predict(X_test2)

print("ensemble Recall:", recall_score(y_test2, y_pred2))
print("Accuracy:", accuracy_score(y_test2, y_pred2))

ensemble Recall: 0.9047619047619048
Accuracy: 0.9230769230769231


In [107]:
# 음성과 텍스트 동시에 학습

tt = pd.DataFrame(X_test1)
X_test3 = X_test.drop(columns='voice_feature')
X_test3.reset_index(drop=True,inplace=True)
X_test3 = pd.concat([tt,X_test3],axis=1)
ttt = pd.DataFrame(X_train1)
X_train3 = X_train.drop(columns='voice_feature')
X_train3.reset_index(drop=True,inplace=True)
X_train3 = pd.concat([ttt,X_train3],axis=1)


y_train3 = y_train
y_test3 = y_test

ss = list(range(90))

X_train3.columns=ss
X_test3.columns=ss


# randomforest
rf_f = RandomForestClassifier(n_estimators=100,n_jobs=-1,max_features=90)
rf_f.fit(X_train3, y_train3)
y_pred_rf = rf_f.predict(X_test3)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test3, y_pred_rf))
print("Recall:", recall_score(y_test3, y_pred_rf))
print(' ')
# gradient boosting
gb_f = GradientBoostingClassifier(n_estimators=100)
gb_f.fit(X_train3, y_train3)
y_pred_gb = gb_f.predict(X_test3)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test3, y_pred_gb))
print("Recall:", recall_score(y_test3, y_pred_gb))
print(' ')
# xgboost
xgb_f = XGBClassifier(n_estimators=100,n_jobs=-1)
xgb_f.fit(X_train3, y_train3)
y_pred_xgb = xgb_f.predict(X_test3)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test3, y_pred_xgb))
print("Recall:", recall_score(y_test3, y_pred_xgb))
print(' ')
# Lightgbm
lgbm_f = LGBMClassifier(n_estimators=100,n_jobs=-1)
lgbm_f.fit(X_train3, y_train3)
y_pred_lgbm = lgbm_f.predict(X_test3)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test3, y_pred_lgbm))
print("Recall:", recall_score(y_test3, y_pred_lgbm))

RandomForest:
Accuracy: 0.9230769230769231
Recall: 0.8809523809523809
 
GradientBoosting:
Accuracy: 0.9120879120879121
Recall: 0.9047619047619048
 
XGBoost:
Accuracy: 0.9230769230769231
Recall: 0.8571428571428571
 
LightGBM:
Accuracy: 0.9010989010989011
Recall: 0.8333333333333334


In [108]:
# 음성과 텍스트 동시 모델 ensemble
ensemble_f = VotingClassifier(estimators=[('rf', rf_f), ('gb', gb_f), ('xgb', xgb_f), ('lgbm', lgbm_f)], voting='soft')
ensemble_f.fit(X_train3, y_train3)
y_pred3 = ensemble_f.predict(X_test3)

print("ensemble Recall:", recall_score(y_test3, y_pred3))
print("Accuracy:", accuracy_score(y_test3, y_pred3))

ensemble Recall: 0.9047619047619048
Accuracy: 0.9340659340659341


In [ ]:
# word2vec

In [ ]:
pip install gensim

In [ ]:
!pip install konlpy

In [113]:
df_sum = pd.concat([voice,normal])
df_sum = df_sum.drop(columns='wav')
df_sum= df_sum.reset_index(drop=True)
df_sum

,txt,voice_fishing
0,지금 전국적으로 약 이백 여명이 넘게 연료가 되어 있는 사건이고 일단 저희도 어느 ...,1
1,금융사기사건이었습니다네.직접 방문하셔서 개설하신.금융권은 어디 금융권들이 있으시죠?...,1
2,그럼 최근 삼 년 사이에 본인개인정보가 담긴 물품 분실하신 적 전혀 없으십니까?제가...,1
3,사실 있는 그대로만 진술하시면 되시는데 현재 계신 곳이 직장 이십니까.통합 불편하시...,1
4,일단 이 두 통장은 범죄로 사용돼서 추가적인 피해자가 발생하지 않도록 본인 의사와 ...,1
...,...,...
177,회원 가입을 했거든요 하신다하는 건가요?혹시 방법을 높여야 하는지 잘 몰라서 보는데...,0
178,근데 이제 장소가 이제 구애가 없다고 하시면 저희가 그을지로 고용노동부건물에 있는 ...,0
179,감사합니다.확인 도와드릴 텐데 방송한 기다려 주시겠어요?여기야 기자 감사드리고요.거...,0
180,네 감사합니다.저 인사담당자신청했는데요 그 a니까 아직도 않았어요?네 감사합니다.,0


In [114]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','요','.','께서','에서','이다','되다']

# 토큰화 
okt = Okt()
tokenized_data = []
for sentence in df_sum['txt']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

# Word2vec 모델 학습
model = Word2Vec(sentences = tokenized_data, window = 5, min_count = 1, workers = 4, sg = 0)
model.wv.vectors.shape

# 각 문장의 벡터 표현 생성
sentence_vectors = []
for text in tokenized_data:
    vector = np.mean([model.wv[word] for word in text], axis=0)
    sentence_vectors.append(vector)

labels = list(df_sum['voice_fishing'])
X_train,X_test,y_train,y_test = train_test_split(sentence_vectors,labels,test_size = 0.5,random_state= 17)


In [145]:
# Word2Vec을 통한 모델 학습

# randomforest
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("RandomForest:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print(' ')
# gradient boosting
gb = GradientBoostingClassifier(n_estimators=100)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
print("GradientBoosting:")
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Recall:", recall_score(y_test, y_pred_gb))
print(' ')
# xgboost
xgb = XGBClassifier(n_estimators=100,n_jobs=-1)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
print("XGBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Recall:", recall_score(y_test, y_pred_xgb))
print(' ')
# Lightgbm
lgbm = LGBMClassifier(n_estimators=100,n_jobs=-1)
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)
print("LightGBM:")
print("Accuracy:", accuracy_score(y_test, y_pred_lgbm))
print("Recall:", recall_score(y_test, y_pred_lgbm))

RandomForest:
Accuracy: 0.8681318681318682
Recall: 0.9354838709677419
 
GradientBoosting:
Accuracy: 0.8571428571428571
Recall: 0.8387096774193549
 
XGBoost:
Accuracy: 0.8571428571428571
Recall: 0.8387096774193549
 
LightGBM:
Accuracy: 0.8681318681318682
Recall: 0.8387096774193549
